In [ ]:
from selenium import webdriver
from konlpy.tag import Okt
from nltk import Text
from matplotlib import font_manager, rc
from wordcloud import WordCloud
from selenium.webdriver.common.by import By
 
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import requests
import time

### url 크롤링

In [ ]:
path = "C:/Users/katenam/chromedriver.exe" # 웹드라이버 실행
 
driver = webdriver.Chrome(path) # 드라이버 경로 설정
url_list_1 = [] # 블로그 url을 저장하기 위한 변수
content_list = "" # 블로그 content를 누적하기 위한 변수

 
for i in range(1, 15):  # 1~2페이지까지의 블로그 내용을 읽어옴
    url = 'https://section.cafe.naver.com/ca-fe/home/search/articles?q=%EB%A1%AF%EB%8D%B0%EC%98%A8&p='+ str(i) + '&pr=7&ps=2020.07.01&pe=2020.08.31'
    driver.get(url)
    time.sleep(0.5) # 오류 방지 sleep
 
    for j in range(1, 3):
        article = '.item_subject'
        articles = driver.find_elements_by_css_selector(article)
        for i in articles:
            article_url = i.get_attribute('href')
            url_list_1.append(article_url)
 
print("url 수집 끝, 해당 url 데이터 크롤링")
print(len(url_list_1))

### url 들어가서 날짜, 내용, 댓글 크롤링

In [ ]:
# 전체 내용 담을 리스트
result_1 = []

for u in tqdm(url_list_1):
    
    driver = webdriver.Chrome('C:/Users/katenam/chromedriver.exe')
    url = (u)
    driver.get(url)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    driver.switch_to.frame("cafe_main")
    container= {}

    # 날짜 크롤링
    date_list = []
    dates = driver.find_elements_by_css_selector('.date')
    for a in dates:
        d = a.text
        date_list.append(d)
    
    # 내용 크롤링
    content_list = []
    content = driver.find_elements_by_css_selector('.se-component.se-text.se-l-default')
    for i in content:
        c = i.text
        content_list.append(c)
    content_str = ' '.join(content_list)
    
    # 댓글 크롤링
    comment_list = []
    comment = driver.find_elements_by_css_selector('.comment_text_box')
    for j in comment:
        co = j.text
        comment_list.append(co)
    comment_str = ' '.join(comment_list)
    

    container['url'] = url
    container['date'] = date_list
    container['content'] = content_str
    container['comment'] = comment_str
    
    result_1.append(container)
    print(container)

    # 각각의 글은 dict라는 딕셔너리에 담음
#     dict[i] = target_info
#     time.sleep(1)

#     # 크롤링 성공하면 글 제목을 출력
#     print(i)

    # 글 하나 크롤링 후 크롬 창 닫기
    driver.close() 

### 크롤링 결과 데이터 프레임으로 저장

In [ ]:
result_df = pd.DataFrame(result_1)
result_df

### 나눠서 크롤링을 했다면 concat을 사용하여 밑으로 이어 붙이기

In [ ]:
result = pd.concat([crawling1,crawling2])
result

### csv로 저장

In [ ]:
result.to_csv("lpay_crawling.csv", index = False)